In [ ]:
import numpy as np

In [ ]:
folder_name = "fake"
num_row = 3
num_col = 3
O_prob = 2
D_prob = 2
cap = 2000
length = 0.5
rhoj = 200
num_lane = 2
spd_lb = 20
spd_ub = 50

assign_horizon = 10
demand = 15

In [ ]:
link_type = 'CTM'
node_type = 'FWJ'
connector_type = 'PQ'
O_type = 'DMOND'
D_type = 'DMDND'

In [ ]:
file_attr = "MNM_input_"
node_file = file_attr + "node"
link_file = file_attr + "link"
snap_file = "Snap_graph"
od_file = file_attr + "od"
demand_file = file_attr + 'demand'

In [ ]:
def add_node_str(node_num, type_str):
    return str(node_num) + ' ' + type_str + '\n'

In [ ]:
def add_snap_str(link_num, from_node, to_node):
    return str(link_num) + ' ' + str(from_node) + ' ' + str(to_node) + '\n';

In [ ]:
def add_link_str(link_num, link_type):
    # CTM 0.8 45 2200 200 1
    # PQ 1 99999 99999 99999 1
    if link_type != 'PQ':
        spd = np.round(np.random.random_sample() * (spd_ub - spd_lb)  + np.float(spd_lb))
        return str(link_num) + ' ' + str(link_type) + ' ' + str(length) + ' ' + str(spd) + ' ' + str(cap) + ' ' + str(rhoj) + ' ' + str(num_lane) + '\n'
    else:
        return str(link_num) + ' PQ 1 99999 99999 99999 1\n'

In [ ]:
def add_OD(od_id, node_id):
    return str(od_id) + ' ' + str(node_id) + '\n'

In [ ]:
link_str_list = []
node_str_list = []
snap_str_list = []
O_dict = dict()
D_dict = dict()
O_str_list = []
D_str_list = []

node_counter = 1
link_counter = 1
O_counter = 1
D_counter = 1

for i in xrange(num_row):
    for j in xrange(num_col):
        node_str_list.append(add_node_str(node_counter, node_type))
        if (j != num_col - 1):
            node1 = num_col * i + j + 1
            node2 = node1 + 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node2))
            link_counter += 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node2, node1))   
            link_counter += 1
        if(i != num_row - 1):
            node1 = num_col * i + j + 1
            node2 = node1 + num_col
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node2))
            link_counter += 1
            link_str_list.append(add_link_str(link_counter, link_type))
            snap_str_list.append(add_snap_str(link_counter, node2, node1)) 
            link_counter += 1
        node_counter += 1
            
for i in xrange(num_row):
    for j in xrange(num_col):
        node1 = num_col * i + j + 1
        if(np.random.random_sample() < O_prob and (j ==0)):
            node_str_list.append(add_node_str(node_counter, O_type))
            O_dict[O_counter] = node1
            O_str_list.append(add_OD(O_counter, node_counter))
            link_str_list.append(add_link_str(link_counter, connector_type))
            snap_str_list.append(add_snap_str(link_counter, node_counter, node1)) 
            link_counter += 1
            node_counter += 1
            O_counter += 1
        if(np.random.random_sample() < D_prob and (j == num_col - 1)):
            node_str_list.append(add_node_str(node_counter, D_type))
            D_dict[D_counter] = node1
            D_str_list.append(add_OD(D_counter, node_counter))
            link_str_list.append(add_link_str(link_counter, connector_type))
            snap_str_list.append(add_snap_str(link_counter, node1, node_counter)) 
            link_counter += 1
            node_counter += 1
            D_counter += 1


In [ ]:
print link_str_list
print node_str_list
print snap_str_list
print O_str_list
print D_str_list
print O_dict

In [ ]:
def add_demand_str(O, D):
    l = [str(O), str(D)]
    d = [str(demand) for i in range(assign_horizon)]
    l = l + d
    return ' '.join(l) + '\n'

In [ ]:
# Origin_ID Destination_ID <demand by interval>
demand_str_list = []

for O, O_node in O_dict.iteritems():
    for D, D_node in D_dict.iteritems():
        if O_node != D_node:
            demand_str_list.append(add_demand_str(O, D))

In [ ]:
f = file(link_file, 'w')
f.write("#ID Type LEN(mile) FFS(mile/h) Cap(v/hour) RHOJ(v/miles) Lane\n")
for line in link_str_list:
    f.write(line)
f.close()

f = file(node_file, 'w')
f.write('#ID Type\n')
for line in node_str_list:
    f.write(line)
f.close()

f = file(snap_file, 'w')
f.write('# EdgeID FromNodeId ToNodeId\n')
for line in snap_str_list:
    f.write(line)
f.close()

f = file(od_file, 'w')
f.write('#Origin_ID <-> node_ID\n')
for line in O_str_list:
    f.write(line)
f.write('#Dest_ID <-> node_ID\n')
for line in D_str_list:
    f.write(line)
f.close()

f = file(demand_file, 'w')
f.write('#Origin_ID Destination_ID <demand by interval>\n')
for line in demand_str_list:
    f.write(line)
f.close()

In [ ]:
print 'maximum_interval:', assign_horizon
print 'num_of_link:', len(link_str_list)
print 'num_of_node:', len(node_str_list)
print 'num_of_O:', len(O_dict)
print 'num_of_D:', len(D_dict)
print 'num_of_od:', len(demand_str_list)